# Ao3 Tags on "Fix-it Fics" as an Indicator of Satisfying Endings #

Are there common themes among fix it fics that show what audiences crave that shows lack? To answer this, I will scrape data from Archive of Our Own (Ao3) to compare tags used on “fix it” fanfictions, or fanfictions that aim to give the show a “better” ending, across the shows Voltron, Supernatural, and Star Wars: The Sequel Trilogy. The answer to this question could be used to suggest what themes audiences are wanting to see in media in general and could be used as a recommendation to media writers on how to create “satisfying” endings. 

##### *** DISCLAIMER *** #####
There are spoilers for each show I will talk about (Voltron, Supernatural, and the Star Wars Sequel Trilogy). All of these shows were completed within recent years. If you are planning to watch these and do not want spoilers, I would not recommend viewing the conclusion section of this report. 

Also, all of these shows, as stated, are finished. I will be giving my recommendations as if the shows have not aired, but the plot is written so the writers could make changes, even though this is impossible as of today.


## Data Collection ##

To do this, I will use the following packages.

In [71]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats

I request data from Ao3 using the requests library. I did some work on Ao3's actual page as it would be easier than generate the url, which I was unsure how to do. There is not a lot of documentation on how Ao3 works, as seen by the lack of API, so all urls will be generated on the actual Ao3 site and then pasted over.



### Playing with the Data ###

I wanted to see how to scrape Ao3, so I used Voltron as a test. I obtained the website url to scrape from by doing a manual search on Ao3's website. Ao3 lists 20 fanfics per page, so this will have 20 fanfics worth of tags. Tags are not limited, so this is entirely dependent on what the author decided to list when publishing the fic. I also decided to exclude any explicit labeled fanfictions for the sake of this being a school project and having to present this data. This works out for the overall mission as well, as all of the fandoms I chose are aired on TV and follow FCC guidelines on how graphic their content can be for TV-14/PG-13.

In [72]:
page = requests.request("GET", 'https://archiveofourown.org/tags/Voltron:%20Legendary%20Defender/works?commit=Sort+and+Filter&exclude_work_search%5Brating_ids%5D%5B%5D=13&page=1&utf8=%E2%9C%93&work_search%5Bcomplete%5D=&work_search%5Bcrossover%5D=&work_search%5Bdate_from%5D=&work_search%5Bdate_to%5D=&work_search%5Bexcluded_tag_names%5D=&work_search%5Blanguage_id%5D=&work_search%5Bother_tag_names%5D=Fix-It&work_search%5Bquery%5D=&work_search%5Bsort_column%5D=revised_at&work_search%5Bwords_from%5D=&work_search%5Bwords_to%5D=')

In [73]:
page

<Response [200]>

In [74]:
soup = bs(page.text, "html.parser")

I looked through the soup in order to find what html tags contained the "tag" items that I will need for my analysis. In this case, it is in linked items (<a/href>) with the class "tag".

In [75]:
tags = soup.find_all('a', class_='tag')

With this, I can clean up the list so it just contains what is inside the tags.

In [76]:
tagList = [x.text for x in soup.find_all('a', class_='tag')]

And with that, I have a list of tags for the first 20 fix-it fanfics from Voltron in Ao3. 


### Function Creation ###
Now that I have learned how to scrape the tags from Ao3, I will create a function that can take the fandom name, a data frame, and how many pages to iterate through as parameters and spit out a data frame of tags for that fandom. The function can be used as long as you use the exact phrase used for the fandom as Ao3 does. If it is your first time using the function, use an empty data frame. In order to keep appending onto that data frame, feed that same data frame into the function when using a different fandom title. The function will a dataframe with two columns: one with the tags and the other with the name of the fandom that tag came from.

The first part of the function is where the showName is changed so the spaces are replaced with "%20", as that is how the url reads spaces. The function updates the showName and then iterates through each page requested, scraping tags from each page. It then goes back through to add the final column of the "Fandom" is came from.

In [77]:
def getTags(showName, df, getPages):
    pageNum = 1
    x = 0
    showNameParse = showName.split()
    showNameAddSpace = ""
    for word in showNameParse:
        showNameAddSpace = showNameAddSpace + showNameParse[x] + "%20"
        x += 1
    #print(showNameAddSpace)
    temp_df = pd.DataFrame()
    while pageNum < getPages:
        page = requests.request("GET", "https://archiveofourown.org/tags/" + showNameAddSpace + "/works?commit=Sort+and+Filter&exclude_work_search%5Brating_ids%5D%5B%5D=13&page=" + str(pageNum) + "&utf8=%E2%9C%93&work_search%5Bcomplete%5D=&work_search%5Bcrossover%5D=&work_search%5Bdate_from%5D=&work_search%5Bdate_to%5D=&work_search%5Bexcluded_tag_names%5D=&work_search%5Blanguage_id%5D=&work_search%5Bother_tag_names%5D=Fix-It&work_search%5Bquery%5D=&work_search%5Bsort_column%5D=revised_at&work_search%5Bwords_from%5D=&work_search%5Bwords_to%5D=")
        soup = bs(page.text, "html.parser")
        tagList = [x.text for x in soup.find_all('a', class_='tag')]
        temp_df = temp_df.append(tagList, ignore_index=True)
        pageNum += 1
        for x in temp_df:
            temp_df["Fandom"] = showName
    df = df.append(temp_df, ignore_index=True)
    return df

### Voltron ###

I chose Voltron for this project due to its notoriety of having a disliked ending by the fans. I will use my function to obtain the tags from the first 10 pages. Originally, I ran the report with 5 pages, which was arbitrary. However, I found that the number of tags pulled from 5 pages was insufficient to find meaningful values in the tag frequency. This will gather tags for the first 200 fanfictions. 

In [78]:
tag_df = pd.DataFrame()
final_tagList = getTags("Voltron: Legendary Defender", tag_df, 10)

### Supernatural ###

Now that I have 100 fanfics worth of tags for Voltron, I will repeat this process for the tv show Supernatural, chosen for its popularity as the subject of fanfiction (over 240,000 fics!) and for its bad ending. 

Since I have already worked out what tags I need and how I can get them from this site, I will be condensing my process to fewer cells with less explanation.

In [ ]:
final_tagList = getTags("Supernatural", final_tagList, 10)

### Star Wars: The Sequel Trilogy ###

This fandom is different than the previous two chosen as it is much bigger than Voltron or even Supernatural since it encompasses many different movies and tv shows. When I chose Star Wars, I did so because I personally did not like the ending to Star Wars Episode IX Rise of Skywalker. I did a preliminary check and found fix-it fanfics for that movie. However, when choosing which fandom website to sort and scrape, I found that there were 11 categories for the Star Wars media franchise. Rather than look through all of the media, which was a category, I chose to stay true to what made me pick Star Wars in general and chose the "Star Wars: The Sequel Trilogy" fandom since it includes Rise of Skywalker.

Despite this being a movie trilogy rather than a tv show, I do not expect this media difference to affect tag usage and preferences among fix-it fanfics. 

In [ ]:
final_tagList = getTags("Star Wars Sequel Trilogy", final_tagList, 10)

In [ ]:
final_tagList.head(15)

## Data Visualization ##

The aspect I am interested in this data is how frequently a certain tag appears within the list. Frequency can be measured a few ways within the data. I will create a pivot table to find and organize the duplicates and then use that information to find the modes within the data. 

In [ ]:
final_tagList = final_tagList.rename(columns={0: "Tags"})
final_tagList.pivot_table(columns=['Tags'], aggfunc ='size')

In [ ]:
final_tag_pivot_both = final_tagList.pivot_table(columns=['Tags', 'Fandom'], aggfunc ='size')
final_tag_pivot_all = final_tagList.pivot_table(columns=['Tags'], aggfunc = 'size')

I created two pivot tables. The first has the columns organized by the tags and the Fandom, so it will count the frequencies of tags for each specific fandom. The pivot_all table ignores fandoms and looks at the frequency of a tag regardless of the fandom it came from.

I attempted to plot this pivot table to get a first look at what the tag frequencies look like. The tables showed that there are some tags that appear much more often than others that I pulled! The goal is to find these tags and see if I can use these tags to generate an idea about what themes people look for in shows in that to be "fixed." However, the table was unreadable due to the sheer amount of values, so I decided not to show it here. I need to filter out all the tags that only have one or two counts in the table.

When I did this, it showed that there were around 800 tags that are used more than once. That is quite a difference from the over 7000 raw tags I pulled and the 3000 tags when duplicates were accounted for. To find the most frequent tags, I will simply increase the number in the comparison until there are 10 or so tags.

In [ ]:
final_tag_pivot_all[final_tag_pivot_all > 65]

In [ ]:
final_tag_pivot_both[final_tag_pivot_both > 55]

These tags are actually not very helpful. Many of the tags are characters from the show, ship names, or just the show itself. I will need to filter these tags out in order to find the most frequent tags that mention specific themes or events happening rather than stating elements already present in the show.

### A Decision ###

I face a dilmena when deciding if I should drop ship names from the tag list. On the one hand, show runners, my intended target audience for my findings, will most likely not take any advice about romantic relationships from a site notorious for throwing together characters seemingly at random sometimes. However, due to the frequency of these tags, it stands to reason that the ships found are legitimate findings that answer my guiding question. It is true to state that creators on Ao3 wanted to see these characters together, and in a few cases, like Voltron, the canon romantic relationships the writers chose to pursue are one of the reasons the fans disliked this ending so much that they felt it needed to be "fixed." However, show writers often have to pander to a diverse audience, making some popular ships on Ao3 an impossibility to pursue as in these examples, these ships are often mlm or wlw. 

However, it can be seen that show runners are influenced by Ao3 creators, as one of the gay ships popularized on Ao3, Destiel, was made canon in the show during its final season. This sets a precedent for shipping popularity to affect a show's direction and writing, so I decided I will keep these tags in my analysis. 

### Cleaning ###
In the following code cells, I will filter out tags of characters, content warnings, and the "fix-it" tags from the dataframe and then present the new pivot tables.

In [ ]:
no_character_df = final_tagList[(final_tagList.Tags != "Dean Winchester") & (final_tagList.Tags != "Keith (Voltron)") & (final_tagList.Tags != "Lance (Voltron)") & (final_tagList.Tags != "Rey (Star Wars)") & (final_tagList.Tags != "Sam Winchester") & (final_tagList.Tags != "Shiro (Voltron)") & (final_tagList.Tags != "Star Wars Sequel Trilogy") & (final_tagList.Tags != "Supernatural") & (final_tagList.Tags != "Voltron: Legendary Defender") & (final_tagList.Tags != "Fix-It") & (final_tagList.Tags != "Alternate Universe - Canon Divergence") & (final_tagList.Tags != "Creator Chose Not To Use Archive Warnings") & (final_tagList.Tags != "No Archive Warnings Apply") & (final_tagList.Tags != "Allura (Voltron)") & (final_tagList.Tags != "Ben Solo | Kylo Ren") & (final_tagList.Tags != "Castiel (Supernatural)") & (final_tagList.Tags != "Hunk (Voltron)") & (final_tagList.Tags != "Pidge | Katie Holt") & (final_tagList.Tags != "Poe Dameron") & (final_tagList.Tags != "Star Wars - All Media Types") & (final_tagList.Tags != "Finn (Star Wars)") & (final_tagList.Tags != "Coran (Voltron)") & (final_tagList.Tags != "Jack Kline") & (final_tagList.Tags != "Leia Organa") & (final_tagList.Tags != "Luke Skywalker") & (final_tagList.Tags != "Rose Tico") & (final_tagList.Tags != "Post-Canon") & (final_tagList.Tags != "Adam (Voltron)") & (final_tagList.Tags != "Post-Canon Fix-It") & (final_tagList.Tags != "Star Wars: The Rise of Skywalker Fix-It") & (final_tagList.Tags != "Eileen Leahy") & (final_tagList.Tags != "Armitage Hux") & (final_tagList.Tags != "Chewbacca (Star Wars)") & (final_tagList.Tags != "Matt Holt") & (final_tagList.Tags != "Fix-It of Sorts") & (final_tagList.Tags != "Romelle (Voltron)") & (final_tagList.Tags != " ") & (final_tagList.Tags != "Ben Solo") & (final_tagList.Tags != "Bobby Singer") & (final_tagList.Tags != "Claire Novak") & (final_tagList.Tags != "Lotor (Voltron)") & (final_tagList.Tags != "Post-Star Wars: The Rise of Skywalker") & (final_tagList.Tags != "Castiel") & (final_tagList.Tags != "Krolia (Voltron)") & (final_tagList.Tags != "Canon Compliant") & (final_tagList.Tags != "Canon-Typical Violence") & (final_tagList.Tags != "Other Additional Tags to Be Added") & (final_tagList.Tags != "Anakin Skywalker") & (final_tagList.Tags != "Graphic Depictions Of Violence") & (final_tagList.Tags != "Han Solo") & (final_tagList.Tags != "Kylo Ren") & (final_tagList.Tags != "Snoke (Star Wars)")]
no_character_df = no_character_df.dropna(axis=0, how="any")

In [ ]:
no_character_pivot_byFandom = no_character_df.pivot_table(columns=['Tags', 'Fandom'], aggfunc ='size')
no_character_pivot_all = no_character_df.pivot_table(columns=['Tags'], aggfunc = 'size')

In [ ]:
no_character_pivot_byFandom[no_character_pivot_byFandom > 25]

In [ ]:
no_character_pivot_all[no_character_pivot_all > 25]

In [ ]:
no_character_Voltron_df = final_tagList[(final_tagList.Tags != "Dean Winchester") & (final_tagList.Tags != "Keith (Voltron)") & (final_tagList.Tags != "Lance (Voltron)") & (final_tagList.Tags != "Rey (Star Wars)") & (final_tagList.Tags != "Sam Winchester") & (final_tagList.Tags != "Shiro (Voltron)") & (final_tagList.Tags != "Star Wars Sequel Trilogy") & (final_tagList.Tags != "Supernatural") & (final_tagList.Tags != "Voltron: Legendary Defender") & (final_tagList.Tags != "Fix-It") & (final_tagList.Tags != "Alternate Universe - Canon Divergence") & (final_tagList.Tags != "Creator Chose Not To Use Archive Warnings") & (final_tagList.Tags != "No Archive Warnings Apply") & (final_tagList.Tags != "Allura (Voltron)") & (final_tagList.Tags != "Ben Solo | Kylo Ren") & (final_tagList.Tags != "Castiel (Supernatural)") & (final_tagList.Tags != "Hunk (Voltron)") & (final_tagList.Tags != "Pidge | Katie Holt") & (final_tagList.Tags != "Poe Dameron") & (final_tagList.Tags != "Star Wars - All Media Types") & (final_tagList.Tags != "Finn (Star Wars)") & (final_tagList.Tags != "Coran (Voltron)") & (final_tagList.Tags != "Jack Kline") & (final_tagList.Tags != "Leia Organa") & (final_tagList.Tags != "Luke Skywalker") & (final_tagList.Tags != "Rose Tico") & (final_tagList.Tags != "Post-Canon") & (final_tagList.Tags != "Adam (Voltron)") & (final_tagList.Tags != "Post-Canon Fix-It") & (final_tagList.Tags != "Star Wars: The Rise of Skywalker Fix-It") & (final_tagList.Tags != "Eileen Leahy") & (final_tagList.Tags != "Armitage Hux") & (final_tagList.Tags != "Chewbacca (Star Wars)") & (final_tagList.Tags != "Matt Holt") & (final_tagList.Tags != "Fix-It of Sorts") & (final_tagList.Tags != "Romelle (Voltron)") & (final_tagList.Tags != " ") & (final_tagList.Tags != "Ben Solo") & (final_tagList.Tags != "Bobby Singer") & (final_tagList.Tags != "Claire Novak") & (final_tagList.Tags != "Lotor (Voltron)") & (final_tagList.Tags != "Post-Star Wars: The Rise of Skywalker") & (final_tagList.Tags != "Castiel") & (final_tagList.Tags != "Krolia (Voltron)") & (final_tagList.Tags != "Canon Compliant") & (final_tagList.Tags != "Canon-Typical Violence") & (final_tagList.Tags != "Other Additional Tags to Be Added") & (final_tagList.Tags != "Anakin Skywalker") & (final_tagList.Tags != "Graphic Depictions Of Violence") & (final_tagList.Tags != "Han Solo") & (final_tagList.Tags != "Kylo Ren") & (final_tagList.Tags != "Snoke (Star Wars)") & (final_tagList.Fandom == "Voltron: Legendary Defender")]
no_character_Supernatural_df = final_tagList[(final_tagList.Tags != "Dean Winchester") & (final_tagList.Tags != "Keith (Voltron)") & (final_tagList.Tags != "Lance (Voltron)") & (final_tagList.Tags != "Rey (Star Wars)") & (final_tagList.Tags != "Sam Winchester") & (final_tagList.Tags != "Shiro (Voltron)") & (final_tagList.Tags != "Star Wars Sequel Trilogy") & (final_tagList.Tags != "Supernatural") & (final_tagList.Tags != "Voltron: Legendary Defender") & (final_tagList.Tags != "Fix-It") & (final_tagList.Tags != "Alternate Universe - Canon Divergence") & (final_tagList.Tags != "Creator Chose Not To Use Archive Warnings") & (final_tagList.Tags != "No Archive Warnings Apply") & (final_tagList.Tags != "Allura (Voltron)") & (final_tagList.Tags != "Ben Solo | Kylo Ren") & (final_tagList.Tags != "Castiel (Supernatural)") & (final_tagList.Tags != "Hunk (Voltron)") & (final_tagList.Tags != "Pidge | Katie Holt") & (final_tagList.Tags != "Poe Dameron") & (final_tagList.Tags != "Star Wars - All Media Types") & (final_tagList.Tags != "Finn (Star Wars)") & (final_tagList.Tags != "Coran (Voltron)") & (final_tagList.Tags != "Jack Kline") & (final_tagList.Tags != "Leia Organa") & (final_tagList.Tags != "Luke Skywalker") & (final_tagList.Tags != "Rose Tico") & (final_tagList.Tags != "Post-Canon") & (final_tagList.Tags != "Adam (Voltron)") & (final_tagList.Tags != "Post-Canon Fix-It") & (final_tagList.Tags != "Star Wars: The Rise of Skywalker Fix-It") & (final_tagList.Tags != "Eileen Leahy") & (final_tagList.Tags != "Armitage Hux") & (final_tagList.Tags != "Chewbacca (Star Wars)") & (final_tagList.Tags != "Matt Holt") & (final_tagList.Tags != "Fix-It of Sorts") & (final_tagList.Tags != "Romelle (Voltron)") & (final_tagList.Tags != " ") & (final_tagList.Tags != "Ben Solo") & (final_tagList.Tags != "Bobby Singer") & (final_tagList.Tags != "Claire Novak") & (final_tagList.Tags != "Lotor (Voltron)") & (final_tagList.Tags != "Post-Star Wars: The Rise of Skywalker") & (final_tagList.Tags != "Castiel") & (final_tagList.Tags != "Krolia (Voltron)") & (final_tagList.Tags != "Canon Compliant") & (final_tagList.Tags != "Canon-Typical Violence") & (final_tagList.Tags != "Other Additional Tags to Be Added") & (final_tagList.Tags != "Anakin Skywalker") & (final_tagList.Tags != "Graphic Depictions Of Violence") & (final_tagList.Tags != "Han Solo") & (final_tagList.Tags != "Kylo Ren") & (final_tagList.Tags != "Snoke (Star Wars)") & (final_tagList.Fandom == "Supernatural")]
no_character_StarWars_df = final_tagList[(final_tagList.Tags != "Dean Winchester") & (final_tagList.Tags != "Keith (Voltron)") & (final_tagList.Tags != "Lance (Voltron)") & (final_tagList.Tags != "Rey (Star Wars)") & (final_tagList.Tags != "Sam Winchester") & (final_tagList.Tags != "Shiro (Voltron)") & (final_tagList.Tags != "Star Wars Sequel Trilogy") & (final_tagList.Tags != "Supernatural") & (final_tagList.Tags != "Voltron: Legendary Defender") & (final_tagList.Tags != "Fix-It") & (final_tagList.Tags != "Alternate Universe - Canon Divergence") & (final_tagList.Tags != "Creator Chose Not To Use Archive Warnings") & (final_tagList.Tags != "No Archive Warnings Apply") & (final_tagList.Tags != "Allura (Voltron)") & (final_tagList.Tags != "Ben Solo | Kylo Ren") & (final_tagList.Tags != "Castiel (Supernatural)") & (final_tagList.Tags != "Hunk (Voltron)") & (final_tagList.Tags != "Pidge | Katie Holt") & (final_tagList.Tags != "Poe Dameron") & (final_tagList.Tags != "Star Wars - All Media Types") & (final_tagList.Tags != "Finn (Star Wars)") & (final_tagList.Tags != "Coran (Voltron)") & (final_tagList.Tags != "Jack Kline") & (final_tagList.Tags != "Leia Organa") & (final_tagList.Tags != "Luke Skywalker") & (final_tagList.Tags != "Rose Tico") & (final_tagList.Tags != "Post-Canon") & (final_tagList.Tags != "Adam (Voltron)") & (final_tagList.Tags != "Post-Canon Fix-It") & (final_tagList.Tags != "Star Wars: The Rise of Skywalker Fix-It") & (final_tagList.Tags != "Eileen Leahy") & (final_tagList.Tags != "Armitage Hux") & (final_tagList.Tags != "Chewbacca (Star Wars)") & (final_tagList.Tags != "Matt Holt") & (final_tagList.Tags != "Fix-It of Sorts") & (final_tagList.Tags != "Romelle (Voltron)") & (final_tagList.Tags != " ") & (final_tagList.Tags != "Ben Solo") & (final_tagList.Tags != "Bobby Singer") & (final_tagList.Tags != "Claire Novak") & (final_tagList.Tags != "Lotor (Voltron)") & (final_tagList.Tags != "Post-Star Wars: The Rise of Skywalker") & (final_tagList.Tags != "Castiel") & (final_tagList.Tags != "Krolia (Voltron)") & (final_tagList.Tags != "Canon Compliant") & (final_tagList.Tags != "Canon-Typical Violence") & (final_tagList.Tags != "Other Additional Tags to Be Added") & (final_tagList.Tags != "Anakin Skywalker") & (final_tagList.Tags != "Graphic Depictions Of Violence") & (final_tagList.Tags != "Han Solo") & (final_tagList.Tags != "Kylo Ren") & (final_tagList.Tags != "Snoke (Star Wars)") & (final_tagList.Fandom == "Star Wars Sequel Trilogy")]


no_character_pivot_Voltron = no_character_Voltron_df.pivot_table(columns=['Tags'], aggfunc = 'size')
no_character_pivot_Supernatural = no_character_Supernatural_df.pivot_table(columns=['Tags'], aggfunc = 'size')
no_character_pivot_StarWars = no_character_StarWars_df.pivot_table(columns=['Tags'], aggfunc = 'size')

In [ ]:
no_character_pivot_Voltron[no_character_pivot_Voltron > 20]

In [ ]:
no_character_pivot_Supernatural[no_character_pivot_Supernatural > 20]

In [ ]:
no_character_pivot_StarWars[no_character_pivot_StarWars > 20]

In [ ]:
no_character_pivot_all[no_character_pivot_all > 30].plot.barh()

In [ ]:
no_character_pivot_Voltron[no_character_pivot_Voltron > 20].plot.barh()

In [ ]:
no_character_pivot_Supernatural[no_character_pivot_Supernatural > 18].plot.barh()

In [ ]:
no_character_pivot_StarWars[no_character_pivot_StarWars > 20].plot.barh()

## Conclusions ##

The most popular tags overall are Angst, Fluff, Hurt/Comfort, and Slow Burn. Ship name tags are also quite popular, with Destiel, Reylo, and Klance being the ships of choice. When looking at each fandom individually, the tags show that fluff and angst are consistent across the fandoms. Happy endings are a common tag shared between Supernatural and Star Wars Sequel Trilogy, while Volton tags are about building to a romance/ unrequited love. 

Through these charts, we are able to see what the creators on Ao3 felt needed to be fixed across these three shows.

### Voltron ###
The Voltron graphs reveals that most creators on Ao3 wrote about the popular ship Keith/Lance (Klance). This ship was not canon in the show, as Lance canonically is interested in Allura. Keith is in 7 out of 15 tags shown on the graph, indicating that he is a popular character. 

The character Adam is also in the tag list a few times. Adam is a character that was added to the show as a romantic interest for Shiro. This was significant as it was the first homosexual relationship to be canonized in the show. Many fans were disappointed by Adam, though, as he was dead and just added to Shiro's backstory. This explains the inclusion of the "Adam lives" tag.

Tags such as hurt/comfort and fluff/angst show that the creators on Ao3 are looking for social interactions between characters rather than action packed sequences. These tags are added to fanfictions where characters work through miscommunications, struggles, and interact on an emotional level. 

Due to these facts, I would recommend to show runner to include more character connections rather than focusing on action. As a fantasy action show, there still needs to be more human connections showcased throughout with real feeling struggles. Based on the tags listed, viewers do not feel satisfied with the current level of character development. Relationships on the show should demonstrate human struggles as well as triumphs to fulfill the emotional needs of the audience. There should also be more plot lines with Keith, as he is a fan favorite, and to consider if it made sense to include the romance between Keith and Lance, as this is the most popular ship. This leads into the last recommendation, which is to handle mlm and wlw relationships on the show with more care, tact, and emotion. The mlm relationship on the show between Adam and Shiro was not well received, as many fanfictions "fixed" their relationship and brought Adam on as a main character rather than a tragic backstory. 


In [ ]:
no_character_pivot_Voltron[no_character_pivot_Voltron > 20].plot.barh()

### Supernatural ###
There is a clear win in 'most popular tag.' The ship tag between Castiel and Dean is by far the most popular, with about 100 more uses than the second most common tag. As discussed previously, the this show actually did include a Castiel/Dean (Destiel) moment in the last season of the show. Similar to Voltron, however, fans were not 100% happy with how this scene played out between these characters. Castiel confesses his love to Dean as Castiel is dying, contributing to the "bury your gays" trope, where many show writers will add in a gay relationship and then immediately kill off one or both of the characters. This may have also lead to the prevalence of "happy ending" tags seen.

The "happy ending" tag may also be coupled with the "s15 ep20 fix-it" tag. Season 15 episode 20 is the last episode of the series, and although I did not watch this episode, I do know there is a major character death, so the tag could be used to reverse this decisions. However, there is no "Dean is alive" tag that is popular, so I am not confident that the character death is what is being "fixed" in these fanfictions. 

Besides some other ships in the show, hurt/comfort and fluff/angst make an appearance in the list. As with Voltron, this indicates more emotions and connections between characters.

Based on these tags, my recommendations to the show runners of Supernatural would be to introduce the connection between Castiel and Dean earlier in the show to allow these feelings to be explored between the characters. To be put in the perspective of an audience member who wanted to see Destiel, you spent the whole 15 seasons of the show seeing a potential connection between these characters. It is assumed that the show will never canonize a romantic connection, but then, in the final season, they reveal that there were romantic intentions with one of the characters. But alas, this character is now dead and the audience is left to deal with the emotions of this confession as well as the emotions that come with a character death. This can feel unsatisfying, and having an earlier reveal could have rectified this. 

In [ ]:
no_character_pivot_Supernatural[no_character_pivot_Supernatural > 18].plot.barh()

### Star Wars Sequel Trilogy ###
The two main plot points being "fixed" in this series have to do with Kylo Ren. Based on the tags seen in the chart below, Ao3 creators wanted to see more Kylo Ren and Rey content as well as having Kylo Ren redeemed. This is another series where the show runners listened to fans and added a canon relationship between Kylo and Rey. But, like in the previous show, this relationship was canonized by the confession of love between both characters as they were in mortal danger and one did die. This fact along with the presence of the angst and fluff tags indicate that people wanted to see this potential relationship grow and develop before the final scenes of the movie, allowing for more human moments and conflict/resolutions to happen and strengthen the connection between these two characters. 

Kylo Ren was redeemed, in a way, at the end of the film by helping Rey defeat the bad guy. However, he was killed in the process. Many tags indicate that fans would have like to have seen Kylo have more opportunities to redeem himself and to interact with the other characters after he had been redeemed. 

My recommendations to the Star Wars writers based on the most popular fix it fics tags would be to allow more time to explore the feelings between Kylo and Rey in canon as well as letting the transition of Kylo from bad guy to good guy take up more time as well. 

In [ ]:
no_character_pivot_StarWars[no_character_pivot_StarWars > 20].plot.barh()

### Limitations ###
My findings and recommendations are severely limited. The main way in which I am limited is in the scope. In order to make recommendations to show runner, I want to be able to say "the majority of the audience wants..." In this report, I am assuming that content creators on Ao3 share the opinions as the majority of audience members, which I have no way to verify. It also highlights people who were unhappy with the ending, as I specfically looked at fix-it fics, and did not compare this with fanfictions that were happy with the endings. I did choose the shows based on what I personally had seen in the audience communities and chose series that had "bad" endings, but I did not back up these choices with data that compared fix-it fics to fictions that were written in canon with the endings. 


### Overall Recommendations ###

In [ ]:
no_character_pivot_all[no_character_pivot_all > 30].plot.barh()

Looking at all of the tags regardless of fandom, I would recommend to show runners to include more character driven content and plot within shows rather than relying on action. With popular tags on fix-it fics being angst, hurt/comfort, and fluff, it is clear that audiences are craving emotional connections between characters and how social consequences can shape relationships. All of these franchises are built on action, but it is clear that action alone will leave audiences wanting and drives people to create their own endings to fill this void.

To prevent your own show from having many fix-it fics written about it, I would recommend balancing action with character driven plot development as well as allowing any potential relationships to have room to grow and develop connections.